### 1. Configuração do ambiente e carregamento de dados

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
# Carregamento da tabela principal
caminho = '../data/raw/dados_brutos.csv'
df_psico = pd.read_csv(caminho)

In [19]:
# Carregamento de dicionários
dic_sid = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_cid_10.csv')
dic_municipio = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_municipio.csv')
dic_geral = pd.read_csv('../data/dicionarios/br_ms_sia_dicionario.csv')

### 2. Entendimento e limpeza de dados

#### 2.1. Limpeza de dados desnecessários

Como a nossa análise vai focar na região, CID, raça/cor, análise temporal e idade. Podemos remover colunas desnecessárias e também nulas

In [5]:
df_psico.columns

Index(['ano', 'mes', 'sigla_uf', 'id_municipio', 'id_estabelecimento_cnes',
       'id_estabelecimento_cnes_familia', 'id_procedimento_ambulatorial',
       'id_servico_especializado', 'id_classificacao_servico',
       'data_inicio_atendimento', 'data_termino_atendimento',
       'permanencia_atendimento', 'motivo_saida_permanencia',
       'data_motivo_saida_permanencia', 'ano_processamento',
       'mes_processamento', 'ano_atendimento', 'mes_atendimento',
       'data_nascimento_paciente', 'id_municipio_residencia_paciente',
       'origem_paciente', 'nacionalidade_paciente', 'tipo_idade',
       'idade_paciente', 'sexo_paciente', 'raca_cor_paciente',
       'etnia_paciente', 'carater_atendimento', 'cid_principal_categoria',
       'cid_principal_subcategoria', 'cid_causas_associadas_categoria',
       'cid_causas_associadas_subcategoria', 'tipo_droga', 'destino_paciente',
       'local_realizacao_atendimento', 'indicador_situacao_rua',
       'indicador_estrategia_familia', 'quant

In [6]:
# Limpeza de nulos
df = df_psico
percentual_nulos = (df.isnull().sum() / len(df)) * 100 # percentual

colunas_com_nulos = percentual_nulos[percentual_nulos > 0].sort_values(ascending=False)

print("Percentual de valores nulos por coluna:")
print(colunas_com_nulos)

Percentual de valores nulos por coluna:
indicador_situacao_rua                100.000000
indicador_estrategia_familia          100.000000
data_motivo_saida_permanencia          98.534204
cid_causas_associadas_categoria        97.656201
cid_causas_associadas_subcategoria     93.852146
data_inicio_atendimento                79.274803
data_termino_atendimento               79.274803
cid_principal_categoria                65.129777
cid_principal_subcategoria             34.870223
dtype: float64


In [11]:
remover = ['indicador_situacao_rua', 'indicador_estrategia_familia', 'data_motivo_saida_permanencia', 'cid_causas_associadas_categoria', 
           'cid_causas_associadas_subcategoria', 'local_realizacao_atendimento', 'quantidade_atendimentos', 'quantidade_pacientes',
           'sigla_uf', 'id_procedimento_ambulatorial', 'nacionalidade_paciente', 'origem_paciente', 'nacionalidade_paciente', 'tipo_idade',
           'id_estabelecimento_cnes', 'id_estabelecimento_cnes_familia', 'id_servico_especializado', 'id_classificacao_servico', 
           'data_inicio_atendimento', 'data_termino_atendimento', 'permanencia_atendimento', 'motivo_saida_permanencia', 'etnia_paciente',
           'carater_atendimento', 'tipo_droga', 'destino_paciente'
           ]

df_psico = df_psico.drop(columns=remover)

In [15]:
# Remoção de anos menores que 2014 e maiores que 2024 (fora da série temporal da análise):
anos_remover = [2012, 2013, 2025]

df_psico = df_psico[~df_psico['ano'].isin(anos_remover)]

In [23]:
df_psico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3412672 entries, 5 to 3717638
Data columns (total 16 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   ano                                int64 
 1   mes                                int64 
 2   ano_processamento                  int64 
 3   mes_processamento                  int64 
 4   ano_atendimento                    int64 
 5   mes_atendimento                    int64 
 6   data_nascimento_paciente           object
 7   idade_paciente                     int64 
 8   sexo_paciente                      object
 9   raca_cor_paciente                  int64 
 10  cid_principal_categoria            object
 11  cid_principal_subcategoria         object
 12  quantidade_produzida_procedimento  int64 
 13  quantidade_aprovada_procedimento   int64 
 14  nome_municipio                     object
 15  nome_municipio_residencia          object
dtypes: int64(10), object(6)
memory usage: 442

#### 

#### 2.2. Junção com dicionários

In [ ]:
# Municípios
dic_nomes = dic_municipio.set_index('id_municipio')['nome']

# adiciona nome dos municípios de atendimento para a tabela inicial
df_psico['nome_municipio'] = df_psico['id_municipio'].map(dic_nomes)

# adiciona nome dos municípios de residencia dos pacientes para a tabela inicial
df_psico['nome_municipio_residencia'] = df_psico['id_municipio_residencia_paciente'].map(dic_nomes)

In [21]:
# remoção de colunas que não serão mais usadas
df_psico = df_psico.drop(columns=['id_municipio', 'id_municipio_residencia_paciente'])

In [22]:
colunas_chave = ['nome_coluna', 'chave']
dic_geral = dic_geral.drop_duplicates(subset=colunas_chave, keep='first')

In [26]:
# adiciona a etnia dos pacientes
raca_cor = dic_geral[dic_geral['nome_coluna'] == 'raca_cor_paciente']
mapa = raca_cor.set_index('chave')['valor']
df_psico['raca_cor_paciente'] = df_psico['raca_cor_paciente'].astype(str).map(mapa)

In [30]:
df_psico.head()

,ano,mes,ano_processamento,mes_processamento,ano_atendimento,mes_atendimento,data_nascimento_paciente,idade_paciente,sexo_paciente,raca_cor_paciente,cid_principal_categoria,cid_principal_subcategoria,quantidade_produzida_procedimento,quantidade_aprovada_procedimento,nome_municipio,nome_municipio_residencia
5,2014,2,2014,2,2014,2,1986-09-09,27,M,sem informacao,NaN,F102,1,1,São José dos Pinhais,São José dos Pinhais
6,2014,2,2014,2,2014,2,1967-01-27,46,M,parda,NaN,F102,1,1,Lapa,Lapa
7,2014,2,2014,2,2014,2,1980-04-09,33,F,parda,NaN,F430,1,1,Piraquara,Piraquara
8,2014,2,2014,2,2014,2,1973-07-14,39,F,parda,F45,NaN,1,1,Piraquara,Piraquara
9,2014,2,2014,2,2014,2,1954-10-23,59,F,parda,F32,NaN,1,1,Piraquara,Piraquara
